## Preliminaries

### Paths

In [1]:
import os
import pathlib
import sys

In [2]:
if not 'google.colab' in str(get_ipython()):
    
    notebooks = os.getcwd()
    parent = str(pathlib.Path(notebooks).parent)
    sys.path.append(parent)

<br>

Warehouse

```python
warehouse = os.path.join(parent, 'warehouse')
if not os.path.exists(warehouse):
    os.makedirs(warehouse)
```

<br>
<br>

### Libraries

In [3]:
%matplotlib inline

import logging
import collections

import numpy as np
import pandas as pd
import xlrd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


<br>
<br>

## Exploration

In [4]:

uri = '../data/catchment/2020 Trust Catchment Populations_Supplementary MSOA Analysis.xlsx'
sheet_name = 'All Admissions'
usecols = ['CatchmentYear', 'msoa', 'TrustCode', 'patients', 'total_patients', 'proportion', 
           'trust_total_catchment', 'msoa_total_catchment']

try:
    populations = pd.read_excel(io=uri, sheet_name=sheet_name, header=0, usecols=usecols)
except RuntimeError as err:
    raise Exception(err)

In [5]:
populations.head()

,CatchmentYear,msoa,TrustCode,patients,total_patients,proportion,trust_total_catchment,msoa_total_catchment
0,2018,E02000001,R1H,963,2598,0.370670,1139498,3241
1,2018,E02000001,RRV,814,2598,0.313318,620885,2631
2,2018,E02000001,RP6,133,2598,0.051193,162590,322
3,2018,E02000001,RQX,158,2598,0.060816,298470,519
4,2018,E02000001,RJ1,241,2598,0.092764,814656,904


In [6]:
populations = populations.loc[populations.CatchmentYear == 2011, :]

In [7]:
populations.reset_index(drop=True, inplace=True)

<br>
<br>

### A Trust

* `np.true_divide(airdale.patients, airdale.total_patients)`

In [8]:
airdale = populations.loc[(populations.TrustCode == 'RCF'), :]

In [9]:
airdale

,CatchmentYear,msoa,TrustCode,patients,total_patients,proportion,trust_total_catchment,msoa_total_catchment
21139,2011,E02002183,RCF,1656,2382,0.695214,161763,3760
21143,2011,E02002184,RCF,3634,5536,0.656431,161763,8234
21150,2011,E02002185,RCF,1148,2249,0.510449,161763,2925
21154,2011,E02002186,RCF,4283,5469,0.783141,161763,9746
21162,2011,E02002187,RCF,515,2070,0.248792,161763,1465
...,...,...,...,...,...,...,...,...
47224,2011,E02005748,RCF,2238,2960,0.756081,161763,5495
47230,2011,E02005749,RCF,3245,4122,0.787239,161763,7474
47350,2011,E02005766,RCF,9,2179,0.004130,161763,24
47409,2011,E02005778,RCF,131,2193,0.059736,161763,383


In [10]:
airdale.msoa_total_catchment.sum()

157668

In [11]:
T = airdale[['msoa', 'patients', 'total_patients']].copy()

In [34]:
T.patients.sum()

66944

<br>
<br>

### 2011 Populations

**URI**

In [12]:
uri = '../data/catchment/mid2011msoaquinaryageestimates.xls'

<br>

**Reading theExcel Files**

In [13]:
class Populations:
    
    def __init__(self, uri, sheet_name):
        """
        
        :param uri:
        """

        # Path
        self.uri = uri
        self.sheet_name = sheet_name

        # Spreadsheet
        Data = collections.namedtuple(typename='Data', field_names=['sheet_name', 'cells', 'start', 'end'])
        self.data = Data._make((self.sheet_name, 'A:W', 5, 7121))

        FieldNames = collections.namedtuple(typename='FieldNames', field_names=['cells', 'row'])
        self.fieldnames = FieldNames._make(('A:W', 4))


    def dataset(self) -> pd.DataFrame:
        """
        :return:
        """

        try:
            return pd.read_excel(io=self.uri, sheet_name=self.data.sheet_name, header=None,
                                 skiprows=np.arange(self.data.start - 1), usecols=self.data.cells,
                                 nrows=(self.data.end - self.data.start + 1))
        except OSError as err:
            raise Exception(err.strerror) from err

    def fields(self) -> list:
        """
        :return:
        """

        try:
            names = pd.read_excel(io=self.uri, sheet_name=self.data.sheet_name, header=None,
                                  skiprows=self.fieldnames.row - 1, usecols=self.fieldnames.cells, nrows=1)
        except OSError as err:
            raise Exception(err.strerror) from err

        return names.astype(str).values.tolist()[0]
    
    def exc(self) -> pd.DataFrame:
        """
        :return: A frame wherein
        """

        # Data
        data: pd.DataFrame = self.dataset()
        data: pd.DataFrame = data.set_axis(labels=self.fields(), axis=1)
        
        return data

<br>

**Males**

In [14]:
males = Populations(uri=uri, sheet_name='Mid-2011 Males').exc()

In [15]:
males.shape

(7117, 23)

In [16]:
males = males.copy().loc[~males['Area Names'].notna(), :]
males.drop(columns=['Area Names', 'nan', 'All Ages'], inplace=True)
males.head()

,Area Codes,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+
1,E02004297,240,191,198,203,181,231,228,287,309,326,291,267,276,234,183,133,82,39,18
2,E02004290,151,150,159,194,146,160,175,174,235,271,230,231,219,161,135,91,46,22,2
3,E02004298,247,244,259,291,225,216,240,300,340,393,361,358,396,289,236,162,107,58,26
4,E02004299,287,218,239,263,250,240,273,253,280,314,301,264,300,235,158,125,95,53,8
5,E02004291,184,177,166,214,184,227,195,228,254,304,222,203,230,157,146,105,58,21,14


In [17]:
males.loc[:, 'sex'] = 'male'

In [18]:
males

,Area Codes,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,...,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+,sex
1,E02004297,240,191,198,203,181,231,228,287,309,...,291,267,276,234,183,133,82,39,18,male
2,E02004290,151,150,159,194,146,160,175,174,235,...,230,231,219,161,135,91,46,22,2,male
3,E02004298,247,244,259,291,225,216,240,300,340,...,361,358,396,289,236,162,107,58,26,male
4,E02004299,287,218,239,263,250,240,273,253,280,...,301,264,300,235,158,125,95,53,8,male
5,E02004291,184,177,166,214,184,227,195,228,254,...,222,203,230,157,146,105,58,21,14,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7112,E02006113,157,157,163,197,192,169,167,145,208,...,219,226,265,262,216,166,124,85,32,male
7113,E02006114,125,117,138,157,232,211,157,116,139,...,217,188,277,250,228,197,135,80,30,male
7114,E02006115,162,167,180,256,173,160,127,136,192,...,251,266,309,260,213,166,113,66,14,male
7115,E02006116,167,181,204,189,144,137,120,147,222,...,277,330,388,392,271,170,138,65,28,male


<br>
<br>

**Females**

In [19]:
females = Populations(uri=uri, sheet_name='Mid-2011 Females').exc()

In [20]:
females.shape

(7117, 23)

In [21]:
females = females.copy().loc[~females['Area Names'].notna(), :]
females.drop(columns=['Area Names', 'nan', 'All Ages'], inplace=True)
females.head()

,Area Codes,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+
1,E02004297,206,155,177,224,185,240,262,268,289,340,321,260,270,241,179,146,111,73,36
2,E02004290,159,172,162,170,141,158,182,204,227,290,216,230,220,185,143,92,42,30,15
3,E02004298,255,222,275,262,180,226,247,325,378,411,397,349,385,272,248,215,137,106,68
4,E02004299,235,194,201,247,244,260,262,253,331,307,314,298,277,233,193,149,133,113,55
5,E02004291,195,178,191,199,176,220,210,205,259,289,251,202,222,185,178,133,111,57,38


In [22]:
females.loc[:, 'sex'] = 'female'

In [23]:
females

,Area Codes,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,...,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+,sex
1,E02004297,206,155,177,224,185,240,262,268,289,...,321,260,270,241,179,146,111,73,36,female
2,E02004290,159,172,162,170,141,158,182,204,227,...,216,230,220,185,143,92,42,30,15,female
3,E02004298,255,222,275,262,180,226,247,325,378,...,397,349,385,272,248,215,137,106,68,female
4,E02004299,235,194,201,247,244,260,262,253,331,...,314,298,277,233,193,149,133,113,55,female
5,E02004291,195,178,191,199,176,220,210,205,259,...,251,202,222,185,178,133,111,57,38,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7112,E02006113,161,153,183,195,226,174,155,182,209,...,240,214,275,273,210,252,206,216,144,female
7113,E02006114,145,120,159,152,220,204,122,136,152,...,215,223,297,275,241,237,182,140,92,female
7114,E02006115,173,146,178,234,175,154,135,173,213,...,274,279,314,289,236,207,150,101,50,female
7115,E02006116,159,164,174,161,131,141,128,158,266,...,288,357,422,393,250,182,153,109,66,female


<br>
<br>

### Calculations

In [24]:
agegroups = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44',
             '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90+']

In [25]:
demographics = pd.concat((females, males), axis=0)
demographics.head()

,Area Codes,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,...,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+,sex
1,E02004297,206,155,177,224,185,240,262,268,289,...,321,260,270,241,179,146,111,73,36,female
2,E02004290,159,172,162,170,141,158,182,204,227,...,216,230,220,185,143,92,42,30,15,female
3,E02004298,255,222,275,262,180,226,247,325,378,...,397,349,385,272,248,215,137,106,68,female
4,E02004299,235,194,201,247,244,260,262,253,331,...,314,298,277,233,193,149,133,113,55,female
5,E02004291,195,178,191,199,176,220,210,205,259,...,251,202,222,185,178,133,111,57,38,female


In [26]:
T.head()

,msoa,patients,total_patients
21139,E02002183,1656,2382
21143,E02002184,3634,5536
21150,E02002185,1148,2249
21154,E02002186,4283,5469
21162,E02002187,515,2070


In [27]:
T.loc[:, 'proportion'] = np.true_divide(T['patients'], T['total_patients'])
T.head()

,msoa,patients,total_patients,proportion
21139,E02002183,1656,2382,0.695214
21143,E02002184,3634,5536,0.656431
21150,E02002185,1148,2249,0.510449
21154,E02002186,4283,5469,0.783141
21162,E02002187,515,2070,0.248792


In [28]:
X = T.merge(demographics, how='left', left_on='msoa', right_on='Area Codes')
X

,msoa,patients,total_patients,proportion,Area Codes,0-4,5-9,10-14,15-19,20-24,...,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+,sex
0,E02002183,1656,2382,0.695214,E02002183,93,120,145,139,81,...,201,192,236,236,185,161,132,92,53,female
1,E02002183,1656,2382,0.695214,E02002183,122,172,180,149,69,...,224,165,233,193,162,142,110,47,17,male
2,E02002184,3634,5536,0.656431,E02002184,348,365,384,392,196,...,472,433,429,372,398,358,340,278,198,female
3,E02002184,3634,5536,0.656431,E02002184,337,385,424,366,230,...,430,403,427,348,292,268,229,131,43,male
4,E02002185,1148,2249,0.510449,E02002185,202,164,182,156,85,...,207,187,224,165,138,133,99,81,26,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,E02005766,9,2179,0.004130,E02005766,135,151,182,155,133,...,245,226,277,211,166,96,68,38,13,male
240,E02005778,131,2193,0.059736,E02005778,185,197,174,153,104,...,228,205,257,230,188,123,84,30,19,female
241,E02005778,131,2193,0.059736,E02005778,156,190,200,160,151,...,234,207,209,205,188,148,83,24,13,male
242,E02006881,15,4350,0.003448,E02006881,313,258,269,329,347,...,360,329,372,235,223,179,130,103,63,female


In [57]:
tallies = X[['msoa'] + agegroups].groupby(by=['msoa']).agg('sum').sum(axis=1)
tallies = tallies.to_frame(name = 'total')
tallies.reset_index(drop=False, inplace=True)
tallies

,msoa,total
0,E02002183,5569
1,E02002184,12904
2,E02002185,5936
3,E02002186,12630
4,E02002187,5881
...,...,...
117,E02005748,7412
118,E02005749,9659
119,E02005766,6115
120,E02005778,6172


In [59]:
baseline = X.merge(tallies, how='left', on='msoa')
baseline

,msoa,patients,total_patients,proportion,Area Codes,0-4,5-9,10-14,15-19,20-24,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+,sex,total
0,E02002183,1656,2382,0.695214,E02002183,93,120,145,139,81,...,192,236,236,185,161,132,92,53,female,5569
1,E02002183,1656,2382,0.695214,E02002183,122,172,180,149,69,...,165,233,193,162,142,110,47,17,male,5569
2,E02002184,3634,5536,0.656431,E02002184,348,365,384,392,196,...,433,429,372,398,358,340,278,198,female,12904
3,E02002184,3634,5536,0.656431,E02002184,337,385,424,366,230,...,403,427,348,292,268,229,131,43,male,12904
4,E02002185,1148,2249,0.510449,E02002185,202,164,182,156,85,...,187,224,165,138,133,99,81,26,female,5936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,E02005766,9,2179,0.004130,E02005766,135,151,182,155,133,...,226,277,211,166,96,68,38,13,male,6115
240,E02005778,131,2193,0.059736,E02005778,185,197,174,153,104,...,205,257,230,188,123,84,30,19,female,6172
241,E02005778,131,2193,0.059736,E02005778,156,190,200,160,151,...,207,209,205,188,148,83,24,13,male,6172
242,E02006881,15,4350,0.003448,E02006881,313,258,269,329,347,...,329,372,235,223,179,130,103,63,female,9948


In [69]:
Z = pd.concat( (baseline[['msoa', 'sex', 'proportion']], np.true_divide(baseline[agegroups], baseline['total'].values.reshape((-1, 1)))), axis=1)
Z = pd.concat( (Z[['msoa', 'sex']], np.multiply(Z[agegroups], Z['proportion'].values.reshape((-1, 1)))), axis=1)
Z

,msoa,sex,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,...,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+
0,E02002183,female,0.011610,0.014980,0.018101,0.017352,0.010112,0.009987,0.009737,0.021472,...,0.032707,0.025092,0.023969,0.029461,0.029461,0.023095,0.020099,0.016478,0.011485,0.006616
1,E02002183,male,0.015230,0.021472,0.022471,0.018601,0.008614,0.008863,0.010361,0.016728,...,0.030086,0.027963,0.020598,0.029087,0.024093,0.020224,0.017727,0.013732,0.005867,0.002122
2,E02002184,female,0.017703,0.018568,0.019534,0.019941,0.009971,0.011090,0.013888,0.019229,...,0.027419,0.024011,0.022027,0.021823,0.018924,0.020246,0.018212,0.017296,0.014142,0.010072
3,E02002184,male,0.017143,0.019585,0.021569,0.018619,0.011700,0.011344,0.013175,0.017245,...,0.023756,0.021874,0.020501,0.021722,0.017703,0.014854,0.013633,0.011649,0.006664,0.002187
4,E02002185,female,0.017370,0.014103,0.015651,0.013415,0.007309,0.008427,0.015049,0.020036,...,0.023820,0.017800,0.016081,0.019262,0.014189,0.011867,0.011437,0.008513,0.006965,0.002236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,E02005766,male,0.000091,0.000102,0.000123,0.000105,0.000090,0.000089,0.000092,0.000113,...,0.000170,0.000165,0.000153,0.000187,0.000143,0.000112,0.000065,0.000046,0.000026,0.000009
240,E02005778,female,0.001791,0.001907,0.001684,0.001481,0.001007,0.001074,0.001181,0.001626,...,0.002749,0.002207,0.001984,0.002487,0.002226,0.001820,0.001190,0.000813,0.000290,0.000184
241,E02005778,male,0.001510,0.001839,0.001936,0.001549,0.001461,0.001084,0.001055,0.001645,...,0.002574,0.002265,0.002003,0.002023,0.001984,0.001820,0.001432,0.000803,0.000232,0.000126
242,E02006881,female,0.000108,0.000089,0.000093,0.000114,0.000120,0.000102,0.000093,0.000114,...,0.000133,0.000125,0.000114,0.000129,0.000081,0.000077,0.000062,0.000045,0.000036,0.000022


In [70]:
np.multiply(Z[agegroups], X['total_patients'].values.reshape((-1, 1))).sum(axis=1)

0       855.505836
1       800.494164
2      1931.618568
3      1702.381432
4       594.693396
          ...     
239       4.410957
240      66.073072
241      64.926928
242       7.747286
243       7.252714
Length: 244, dtype: float64

<br>
<br>

## MSOA: Bradford 001

A MSOA & Year

In [ ]:
bradford0001 = populations.loc[(populations.msoa == 'E02002183') & (populations.CatchmentYear == 2011), :]
bradford0001

In [ ]:
bradford0001.patients.sum()

In [ ]:
np.true_divide(bradford0001.patients, bradford0001.total_patients)

<br>

Males

In [ ]:
M = males.loc[males['Area Codes'] == 'E02002183', :]
M

In [ ]:
M.set_index(pd.Index(['male']), inplace=True)

<br>

Females

In [ ]:
F = females.loc[females['Area Codes'] == 'E02002183', :]
F

In [ ]:
F.set_index(pd.Index(['female']), inplace=True)

<br>

Attributions

In [ ]:
bradford0001

<br>

Collating F & M cells

In [ ]:
people = pd.concat((F, M), axis=0)
people

<br>

Total Bradford 001 Population

In [ ]:
people.iloc[:, 3:].sum(axis=1)

In [ ]:
people['All Ages'].sum()

<br>

Tests

In [ ]:
matrix = people.iloc[:, 3:]
matrix

In [ ]:
reference = matrix.sum(axis=1).values.reshape((2,1))
reference

In [ ]:
matrix_ = np.true_divide(matrix, reference)
matrix_

In [ ]:
matrix_ = 0.695214 * matrix_
matrix_